In [2]:
import os
import glob
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Audio
import librosa
import librosa.display

# Scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight

# Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

dataset = []

label_replacements = {
    "20240505": "abnormal",
    "20240417": "normal"
}
for folder in [r"\Users\User\heartbeat\heartBeat-project\input\set_c\**",r"\Users\User\heartbeat\heartBeat-project\input\set_d\**"]:
    for filename in glob.iglob(folder):
        if os.path.exists(filename):
            label = os.path.basename(filename).split("_")[0]
            label = label_replacements.get(label, label)
            duration = librosa.get_duration(filename=filename)
            # skip audio smaller than 3 secs
            if duration>=3:
                slice_size = 3
                iterations = int((duration-slice_size)/(slice_size-1))
                iterations += 1
#                 initial_offset = (duration % slice_size)/2
                initial_offset = (duration - ((iterations*(slice_size-1))+1))/2
                # if label not in ["Aunlabelledtest", "Bunlabelledtest", "artifact"]:
                for i in range(iterations):
                    offset = initial_offset + i*(slice_size-1)
                    if (label == "normal"):
                        dataset.append({
                                "filename": filename,
                                "label": "normal",
                                "offset": offset
                            })
                    else:
                        dataset.append({
                                "filename": filename,
                                "label": "abnormal",
                                "offset": offset
                            })
                        
dataset = pd.DataFrame(dataset)
dataset = shuffle(dataset, random_state=42)
dataset.info()

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

print("Train: %i" % len(train))
print("Test: %i" % len(test))

def extract_features(audio_path,offset):
#     y, sr = librosa.load(audio_path, duration=3)
    y, sr = librosa.load(audio_path, offset=offset, duration=3)
#     y = librosa.util.normalize(y)
    
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, 
                                   hop_length=512, 
                                   n_mels=128)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)

#     mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return mfccs

x_train = []
x_test = []

for idx in tqdm(range(len(train))):
    x_train.append(extract_features(train.filename.iloc[idx],train.offset.iloc[idx]))

for idx in tqdm(range(len(test))):
    x_test.append(extract_features(test.filename.iloc[idx],test.offset.iloc[idx]))
    
x_test = np.asarray(x_test)
x_train = np.asarray(x_train)

print("X train:", x_train.shape)
print("X test:", x_test.shape)

# Encode Labels
encoder = LabelEncoder()
encoder.fit(train.label)

y_train = encoder.transform(train.label)
y_test = encoder.transform(test.label)

# Compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

# Convert the result to a dictionary
class_weights_dict = dict(enumerate(class_weights))

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("X train:", x_train.shape)
print("Y train:", y_train.shape)
print("X test:", x_test.shape)
print("Y test:", y_test.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 524 entries, 310 to 102
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  524 non-null    object 
 1   label     524 non-null    object 
 2   offset    524 non-null    float64
dtypes: float64(1), object(2)
memory usage: 16.4+ KB
Train: 419
Test: 105


100%|██████████| 105/105 [00:02<00:00, 41.24it/s]


X train: (419, 40, 130)
X test: (105, 40, 130)
X train: (419, 40, 130, 1)
Y train: (419, 2)
X test: (105, 40, 130, 1)
Y test: (105, 2)


In [7]:
from moviepy.editor import AudioFileClip
import os

def convert_mp4_audio_to_wav(input_file, output_file):
    # Load the audio clip
    audio_clip = AudioFileClip(input_file)
    
    # Write the audio clip to a WAV file
    audio_clip.write_audiofile(output_file)
    
    # Close the audio clip
    audio_clip.close()

# File paths
input_file = input("input the address of an audio file .. i.e. input/WhatsApp Audio 2024-05-20 at 9.37.19 PM.mpeg: ")
print(input_file)
output_file = "input/heartbeat_audio.wav"

# Convert .mp4 to .wav
convert_mp4_audio_to_wav(input_file, output_file)

print("Conversion complete!")

from keras.models import load_model

# load model
model = load_model("heartbeat_classifier (normalised).h5")

# File to be classified
# classify_file = "input/set_d/20240417_223344_005.wav" #20240417 normal

classify_file = "input/heartbeat_audio.wav"
x_test = []
x_test.append(extract_features(classify_file, 0.5))
x_test = np.asarray(x_test)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
pred = model.predict(x_test,verbose=1)

print(pred)

# Get the class index with the highest probability
pred_class_index = np.argmax(pred)

# Map the class index to the actual class label using the encoder
classes = encoder.classes_
pred_class_label = classes[pred_class_index]

# Print the predicted class and confidence
if pred_class_label == "normal":
    print("Normal heartbeat")
    print("Confidence:", pred[0][pred_class_index])
elif pred_class_label == "abnormal":
    print("Abnormal heartbeat")
    print("Confidence:", pred[0][pred_class_index])
else:
    print("no hearbeat")


input\abnormal-test\20240505_180655.m4a
MoviePy - Writing audio in input/heartbeat_audio.wav


MoviePy - Done.
Conversion complete!
1/1 [==============================] - 0s 115ms/step
[[9.9992716e-01 7.2869763e-05]]
Abnormal heartbeat
Confidence: 0.99992716
